In [522]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 

import croniter
import pandas as pd
import datetime
from datetime import timedelta
import re

# Misc

In [1769]:
now = datetime.datetime.now()
# iter = croniter.croniter('55 23 * * Sat,Mon-Fri 55', pd.Timestamp('2023-08-25'))
iter = croniter.croniter('* */5 * * * 55', pd.Timestamp('2023-08-25 12:00'))

print(pd.Timestamp(iter.get_prev(), unit='s'))
print('- - - -')
for _ in range(10):
    print(pd.Timestamp(iter.get_next(), unit='s'))

2023-08-25 10:59:55
- - - -
2023-08-25 15:00:55
2023-08-25 15:01:55
2023-08-25 15:02:55
2023-08-25 15:03:55
2023-08-25 15:04:55
2023-08-25 15:05:55
2023-08-25 15:06:55
2023-08-25 15:07:55
2023-08-25 15:08:55
2023-08-25 15:09:55


2023-08-25 12:10:55
2023-08-25 12:20:55
2023-08-25 12:30:55
2023-08-25 12:40:55
2023-08-25 12:50:55


In [ ]:
# # cmp = pd.Timedelta("6d23h55min").components
# cmp = pd.Timedelta("5Min").components
# sched = f"{cmp.minutes} {cmp.hours} */{cmp.days} * * {cmp.seconds}"
# iter = croniter.croniter(sched, pd.Timestamp('2023-08-25 12:00'))

# pd.Timestamp(iter.get_prev(), unit='s'), pd.Timestamp(iter.get_next(), unit='s')

In [838]:
from dateutil.parser import parser
from dateutil.rrule import rrulestr
# list(rrulestr("""RRULE:FREQ=DAILY;INTERVAL=10"""))
list(rrulestr('FREQ=WEEKLY;INTERVAL=1;BYHOUR=23;BYMINUTE=45;BYSECOND=0;COUNT=5'))

list(rrulestr('FREQ=WEEKLY;INTERVAL=1;BYHOUR=23;BYMINUTE=45;BYSECOND=0;COUNT=5'))

# Spec parser

In [1787]:
%load_ext autoreload
%autoreload 2
from qubx.core.helpers import BasicScheduler
from qubx.utils.time import convert_seconds_to_str

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1783]:
specs = [
    '',
    'bar.1m: -5Sec',        # 5 sec before subsciption bar closing
    '1Min -5Sec',           # 5 sec before subsciption bar closing
    'bar:  5Sec',           # 5 sec after subsciption bar opened
    'bar:  +5Sec',          # 5 sec after subsciption bar opened
    'time: 23:59:00 @ Sat', # at 23:59:00 every Saturday
    'time: 23:59 @ Sat',    # at 23:59:00 every Saturday
    'time: 9:30:10',        # every day at 9:30:10
    '* * * * * 45',         # raw cron 
    'cron: * * * * * 45',   # raw cron witch implicit type
    '23:59:00 @ Sun',       # at 23:59:00 every Sunday
    '9:30:10',              # every day at 9:30:10
    '9:30 @ Mon-Fri',       # mon, ... fri at 9:30
    '9:30:22 @ Mon,Fri',    # mon and fri at 9:30:22
    '9:30 , 15:45 @ Mon-Fri', # mon, ... fri at 9:30 and 15:45
    '9:30, 15:45 @ Mon, Fri', # mon and fri at 9:30 and 15:45
    '9:30:10  15:45 @ Mon, Fri', # mon and fri at 9:30 and 15:45
    'calendar: 23:00 liquidation', # custom event
    '9:30 -1Min'            # time 
    '-5Sec',                # 5 sec before subsciption bar end
    '5Min -5Sec',           # 5 sec before every 5min interval ends
    '1w -1h',               # 1h before every week ends
    '4month -5hour',        # 5h before the end of every 4 months 
    '1week -3hours -10Mins',# 5h before the end of every 4 months 
    '2h30m -5Sec',          # 5 sec before every 2h and 30min interval ends
]

In [1794]:
pd.Timedelta('12s').as_unit('s').to_timedelta64()

numpy.timedelta64(12,'s')

In [1788]:
bs = BasicScheduler()
for s in specs:
    print(s.ljust(30), bs._process_spec_dict(bs.parse_schedule_spec(s)))

                               []
bar.1m: -5Sec                  [{'type': 'bar', 'args': '-5Sec', 'timeframe': '1m', 'delay': Timedelta('-1 days +23:59:55')}]
1Min -5Sec                     [{'type': 'bar', 'args': '1Min -5Sec', 'timeframe': '1Min', 'delay': Timedelta('-1 days +23:59:55')}]
bar:  5Sec                     [{'type': 'bar', 'args': '5Sec', 'timeframe': None, 'delay': Timedelta('0 days 00:00:05')}]
bar:  +5Sec                    [{'type': 'bar', 'args': '+5Sec', 'timeframe': None, 'delay': Timedelta('0 days 00:00:05')}]
time: 23:59:00 @ Sat           [{'type': 'cron', 'args': '59 23 * * Sat'}]
time: 23:59 @ Sat              [{'type': 'cron', 'args': '59 23 * * Sat'}]
time: 9:30:10                  [{'type': 'cron', 'args': '30 9 * * * 10'}]
* * * * * 45                   [{'type': 'cron', 'args': '* * * * * 45'}]
cron: * * * * * 45             [{'type': 'cron', 'args': '* * * * * 45'}]
23:59:00 @ Sun                 [{'type': 'cron', 'args': '59 23 * * Sun'}]
9:30:10     

In [1726]:
for s in specs: 
    m = re.compile(
        r"((?P<type>[A-Za-z]+)(\.?(?P<timeframe>[0-9A-Za-z]+))?\ *:)?"
        r"\ *"
        r"((?P<spec>"
            r"(?P<time>((\d+:\d+(:\d+)?)\ *,?\ *)+)?"
            r"((\ *@\ *)(?P<by>([A-Za-z0-9-,\ ]+)))?"
            r"(("
            r'((?P<months>[-+]?\d+)(months|month|bm|mo))?'
            r'((?P<weeks>[-+]?\d+)(weeks|week|w))?'
            r'((?P<days>[-+]?\d+)(days|day|d))?'
            r'((?P<hours>[-+]?\d+)(hours|hour|h))?'
            r'((?P<minutes>[-+]?\d+)(mins|min|m))?'
            r'((?P<seconds>[-+]?\d+)(sec|s))?'
            r")(\ *)?)*"
            r".*"
        r"))?"

        , re.IGNORECASE
    ).match(s)
    print(s.ljust(30), '->', {k: v for k, v in m.groupdict().items() if v} if m else '?????')


                               -> {}
bar.1m: -5Sec                  -> {'type': 'bar', 'timeframe': '1m', 'spec': '-5Sec', 'seconds': '-5'}
bar:  5Sec                     -> {'type': 'bar', 'spec': '5Sec', 'seconds': '5'}
bar:  +5Sec                    -> {'type': 'bar', 'spec': '+5Sec', 'seconds': '+5'}
time: 23:59:00 @ Sat           -> {'type': 'time', 'spec': '23:59:00 @ Sat', 'time': '23:59:00 ', 'by': 'Sat'}
time: 23:59 @ Sat              -> {'type': 'time', 'spec': '23:59 @ Sat', 'time': '23:59 ', 'by': 'Sat'}
time: 9:30:10                  -> {'type': 'time', 'spec': '9:30:10', 'time': '9:30:10'}
* * * * * 45                   -> {'spec': '* * * * * 45'}
cron: * * * * * 45             -> {'type': 'cron', 'spec': '* * * * * 45'}
23:59:00 @ Sun                 -> {'spec': '23:59:00 @ Sun', 'time': '23:59:00 ', 'by': 'Sun'}
9:30:10                        -> {'spec': '9:30:10', 'time': '9:30:10'}
9:30 @ Mon-Fri                 -> {'spec': '9:30 @ Mon-Fri', 'time': '9:30 ', 'by': 'Mo

In [1731]:
from typing import Dict, List


if 1:
    S = lambda s: [x for x in re.split(r"[, ]", s) if x]
    HMS = lambda s: list(map(int, s.split(':') if s.count(':') == 2 else [*s.split(':'), 0]))
    AS_INT = lambda d, k: int(d.get(k, 0)) 
    AS_TD = lambda d: pd.Timedelta(d)

    def _make_shift(_b, _w, _d, _h, _m, _s):
        return AS_TD(f'{_b*4}W') + AS_TD(f'{_w}W') + AS_TD(f'{_d}D') + AS_TD(f'{_h}h') + AS_TD(f'{_m}Min') + AS_TD(f'{_s}Sec')

    def _mk_cron(time: str, by: list | None) -> str:
        h,m,s = HMS(time)
        assert h < 24, f'Wrong value for hour {h}'
        assert m < 60, f'Wrong value for minute {m}'
        assert s < 60, f'Wrong value for seconds {s}'
        b = ','.join(by) if by else '*'
        c = f'{m} {h} * * {b}'
        return c if s == 0 else c + f' {s}'

    def _process_spec_dict(spec: dict) -> List[Dict]:
        config = []
        _T, _S = spec.get('type'), spec.get('spec')
        _F = spec.get('timeframe')
        _t, _by = S(spec.get('time', '')), S(spec.get('by', ''))
        _b, _w, _d = AS_INT(spec, 'months'), AS_INT(spec, 'weeks'), AS_INT(spec, 'days')
        _h, _m, _s = AS_INT(spec, 'hours'), AS_INT(spec, 'minutes'), AS_INT(spec, 'seconds')
        _has_intervals = (_b != 0) or (_w != 0) or (_d != 0) or (_h != 0) or (_m != 0) or (_s != 0) 
        _shift = _make_shift(_b, _w, _d, _h, _m, _s)

        match _T:
            case 'cron':
                if not _S or croniter.croniter.is_valid(_S):
                    config.append(dict(type='cron', args=_S))
                else:
                    raise ValueError(f"Wrong specification for cron type: {_S}")

            case 'time':
                for t in _t:
                    config.append(dict(type='cron', args=_mk_cron(t, _by)))
            
            case None:
                if _t: # - if time specified
                    for t in _t:
                        config.append(dict(type='cron', args=_mk_cron(t, _by)))
            case _:
                config.append(dict(type=_T, args=_S, timeframe=_F, delay=_shift))

        return config

In [1692]:
_process_spec_dict( {'spec': '1h -5s', 'hours': '12'})

0 days 12:00:00


[]

In [1693]:
_process_spec_dict( {'spec': '9:30:10  15:45 @ Mon, Fri', 'time': '9:30:10  15:45 ', 'by': 'Mon, Fri'})

[{'type': 'cron', 'args': '30 9 * * Mon,Fri 10'},
 {'type': 'cron', 'args': '45 15 * * Mon,Fri'}]

In [1694]:
_process_spec_dict( {'type': 'time', 'spec': '9:30:10', 'time': '9:30:10'})

[{'type': 'cron', 'args': '30 9 * * * 10'}]

In [1695]:
_process_spec_dict( {'type': 'cron', 'spec': '5 * * * *'})

[{'type': 'cron', 'args': '5 * * * *'}]

In [1732]:
_process_spec_dict( {'type': 'bar', 'spec': '', 'timeframe': '5m', 'minutes': -1})

[{'type': 'bar',
  'args': '',
  'timeframe': '5m',
  'delay': Timedelta('-1 days +23:59:00')}]